# Solving the problem of Γ ⊨ A using CDCL

#### The problem:
* $\Gamma \subseteq_w F_{\mathcal{L}}$ is a Theory (a Finite Set of Propositional Formulas);
* $A \in F_{\mathcal{L}}$ is a Propositional Formula;
* We are asked to determine whether $A$ is a Logical Consequence (⊨) of Γ (equivalently, if Γ "models" A). That is:

$$
\begin{aligned}
\Gamma \vDash A
\end{aligned}
$$

#### The procedure:
1) **Preprocessing**: Transform $\Gamma \vDash A$ into an *equisatisfiable* Propositional Formula in Conjunctive Normal Form (CNF);
2) **Verify**: Evaluate *unsatisfiability* of the CNF using a CNF-SAT Solver (**CDCL**)


In [1]:
from preprocessing import *
from representation import ast, dimacs
from sat_solvers import CDCL, CDCLOptions, HeuristicType

## 1) Load the Problem from a File

The file should contain:
* N lines representing the formulas of the Theory Γ
* An empty, separation line
* The formula A

Example:
```
!(A & B) | (C -> D) -> E
A -> D
!B | (E -> F)

(A & C) -> D
```

Connectives can be expressed using either ASCII characters (!, &, |, ->) or Unicode characters (¬, ∧, ∨, →).

Propositional letters can be any string, although the parser will convert them into integer numbers.

In [2]:
filepath = 'examples/logcons/000.txt'

problem = LogConsProblem.from_file(filepath)
print(problem)

{
 rains -> umbrella
 !umbrella
} ⊨	!rains



## 2) Preprocessing

Preprocessing consists of the following steps:
1) Express the problem $\Gamma \vDash A$ as the formula:
$$
\begin{aligned}
F = \Gamma_1 \wedge \Gamma_2 \wedge ... \wedge \Gamma_N \wedge \lnot A
\end{aligned}
$$
2) Convert the formula $F$ into an *equisatisfiable* CNF. This requires multiple smaller steps:
    1. Convert $F$ into $F^{IFNF} \in IFNF$, an *equivalent* formula in Implication Free Normal Form;
    2. Convert $F^{IFNF}$ into $F^{NNF} \in NNF$, an *equivalent* formula in Negation Normal Form;
    3. Convert $F^{NNF}$ into $F^{CNF} \in CNF$, an *equisatisfiable* formula in Conjunctive Normal Form.
$$
\begin{aligned}
\Gamma \vDash A \to F \to F^{IFNF} \to F^{NNF} \to F^{CNF}
\end{aligned}
$$

In [3]:
formula = problem.to_formula()

formula

'(rains -> umbrella) ∧ (!umbrella) ∧ ¬(!rains)'

In [4]:
f = ast.PropFormula.from_string(formula)
print(f"Input:\t{f}")
f2 = IFNF.to_IFNF(f)
print(f"IFNF:\t{f2}")
f3 = NNF.to_NNF(f2)
print(f"NNF:\t{f3}")
f4 = CNF.to_equisat_CNF(f3)
print(f"Equisat CNF:\t{f4}")
cnf = dimacs.DimacsCNF.from_ast(f4)
print(f"Dimacs CNF:\t{cnf}")

Input:	(((1 → 2) ∧ (¬2)) ∧ (¬(¬1)))
IFNF:	((((¬1) ∨ 2) ∧ (¬2)) ∧ (¬(¬1)))
NNF:	((((¬1) ∨ 2) ∧ (¬2)) ∧ 1)
Equisat CNF:	((((¬1) ∨ 2) ∧ (¬2)) ∧ 1)
Dimacs CNF:	[-1, 2], [-2], [1]


## 3) Apply CDCL

Finally, we can run the **CDCL** procedure to determine whether $F^{CNF}$ is **SATISFIABLE** or **UNSATISFIABLE**.

For practicality, we handle $F^{CNF}$ as a set $S$ of clauses (which themselves are sets of literals).

The **CDCL** procedure will either:
1. Declare $S$ **SAT**, by finding a truth-value assignment satisfying $S$; or
2. Declare $S$ **UNSAT**, by finding a level-0 conflict among the clauses of $S$ and the clauses learnt during the process; or
3. Timeout, if the execution lasts more than the specified `timeout_seconds`.

**In order for $\Gamma \vDash A$, $S$ should be UNSATISFIABLE.**

In [6]:
try:
    cdcl = CDCL(
        cnf=cnf,
        options=CDCLOptions(
            heuristic=HeuristicType.VSIDS,
            restarts=True,
            forgets=True,
            timeout_seconds=120
        )
    )

    found = cdcl.solve()
except Exception as e:
    print(e)
    found = False

if found:
    assert cdcl.cnf.check(cdcl.v) # Verify that the found assignment satisfies the formula
    print(f"CNF is SAT:\n{cdcl.v}")
    print(f"Γ ⊭ A")
else:
    print("CNF is UNSAT")
    print(f"Γ ⊨ A")

There are not enough watchable literals to satisfy the clause clause [-1, 2]
CNF is UNSAT
Γ ⊨ A
